# 이미지 분류 멀티모달 LLM 파인튜닝 튜토리얼

이 튜토리얼에서는 **vLLM**과 **LLaMA-Factory**를 활용하여 멀티모달 LLM(MLLM)을 이미지 분류 작업에 특화되도록 파인튜닝하는 방법을 학습합니다.

## 1. vLLM 서버 실행 및 연결 테스트

먼저 **Qwen2.5-VL-7B-Instruct** 모델을 사용하여 vLLM 서버를 실행합니다. 이 모델은 텍스트와 이미지를 모두 처리할 수 있는 멀티모달 모델입니다.

### 사전 준비사항
1. 🤗 Hugging Face에 가입하여 액세스 토큰을 발급받으세요
2. [Qwen2.5-VL-7B-Instruct 페이지](https://huggingface.co/Qwen/Qwen2.5-VL-7B-Instruct)에서 모델을 확인하세요

### vLLM 서버 실행 명령어 설명
- `HF_TOKEN`: 발급받은 Hugging Face 토큰을 입력하세요
- `CUDA_VISIBLE_DEVICES`: 사용할 GPU 번호 (전체 GPU 사용시 생략 가능)  
- `-tp`: 텐서 병렬화에 사용할 GPU 개수를 지정합니다

**⚠️ 중요**: 아래 명령어를 별도의 터미널에서 실행하세요.

In [ ]:
export HF_TOKEN=<허깅페이스 토큰>
CUDA_VISIBLE_DEVICES=0,1 vllm serve Qwen/Qwen2.5-VL-7B-Instruct -tp 2

In [ ]:
# langchain-openai 라이브러리 임포트
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    base_url="http://localhost:8000/v1",  # vLLM 서버 주소
    api_key="EMPTY",  # vLLM에서는 임의의 값 사용
    model="Qwen/Qwen2.5-VL-7B-Instruct",  # 모델 이름
    temperature=0.1
)

# 이미지 파일을 base64로 인코딩
import base64
from PIL import Image
from io import BytesIO

## data/image_classifiion/input.jpg를 활용해보자.
image = Image.open("data/image_classification/input.jpg")
buffer = BytesIO()
image.save(buffer, format="JPEG")
base64_image = base64.b64encode(buffer.getvalue()).decode("utf-8")

# 채팅 메시지 보내기
from langchain_core.messages import HumanMessage

messages = [HumanMessage(
    content=[
        {"type": "text", "text": "다음 이미지를 설명해줘:"},
        {
            "type": "image_url",
            "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}
        }
    ]
)]
response = llm.invoke(messages)

print("AI 응답:", response.content)

### ✅ vLLM 서버 실행 확인

터미널에서 다음과 같은 로그가 출력되면 vLLM 서버가 정상적으로 시작된 것입니다:

```log
INFO:     Started server process [2609659]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
```

이제 **Langchain**을 사용하여 실행 중인 vLLM 서버에 이미지를 포함한 멀티모달 요청을 보내 정상 작동을 확인해보겠습니다.

### 📊 서버 요청 처리 확인

위 코드를 실행한 후, vLLM을 실행시킨 터미널에서 다음과 같은 로그가 출력되면 멀티모달 요청이 성공적으로 처리된 것입니다:

```log
INFO 08-13 23:26:51 [logger.py:41] Received request chatcmpl-543715af49a8443da65b81033b0cd2de: 
prompt: '<|im_start|>user\n<|vision_start|><|image_pad|><|vision_end|>다음 이미지를 설명해줘:<|im_end|>...
INFO 08-13 23:26:51 [async_llm.py:269] Added request chatcmpl-543715af49a8443da65b81033b0cd2de.
INFO:     127.0.0.1:39490 - "POST /v1/chat/completions HTTP/1.1" 200 OK
```

## 2. 실습 데이터셋 및 목표 설정

### 📋 사용 데이터셋
이번 실습에서는 [sfarrukhm/intel-image-classification](https://huggingface.co/datasets/sfarrukhm/intel-image-classification) 데이터셋을 활용합니다.

### 🎯 실습 목표
자연 이미지를 6개 카테고리로 분류하는 멀티모달 모델을 구축하는 것이 목표입니다.

**분류 카테고리:**
- **Mountain** (산)
- **Glacier** (빙하) 
- **Street** (거리)
- **Sea** (바다)
- **Forest** (숲)
- **Buildings** (건물)

먼저 샘플 이미지를 확인해보겠습니다.

## 3. Zero-shot 정확도 평가

### 🧪 실험 1: 기본 Zero-shot 예측

먼저 **fine-tuning 없이** 기본 Qwen2.5-VL 모델이 이미지 분류를 얼마나 잘 수행할 수 있는지 확인해보겠습니다.

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage

system_prompt = "주어진 이미지에 대해서 다음 중 하나로 분류하시오: ['Mountain', 'Glacier', 'Street', 'Sea', 'Forest', 'Buildings']"

messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(
        content=[{
            "type": "image_url",
            "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}
        }]
    )
]

# LLM에 요청
response = llm.invoke(messages)

print(response.content)

### 🔧 실험 2: Constrained Decoding으로 Zero-shot 예측

기본 zero-shot 출력에서는 다음과 같은 문제가 발생할 수 있습니다:
- 정해진 6개 카테고리 외의 답변 생성
- 불필요한 설명이 포함된 긴 응답
- 일관성 없는 출력 형태

이러한 문제를 해결하기 위해 **Constrained Decoding**을 사용하여 출력을 6개 카테고리 중 하나로 강제로 제한해보겠습니다.

In [ ]:
options = ['Mountain', 'Glacier', 'Street', 'Sea', 'Forest', 'Buildings']

constrained_llm = ChatOpenAI(
    base_url="http://localhost:8000/v1",  # vLLM 서버 주소
    api_key="EMPTY",  # vLLM에서는 임의의 값 사용
    model="Qwen/Qwen2.5-VL-7B-Instruct",  # 모델 이름
    temperature=0.1,
    extra_body={
        "guided_choice": options
    }
)

# LLM에 요청
response = constrained_llm.invoke(messages)

print(response.content)

이제 **Fine-tuning**을 통해 이미지 분류 성능을 개선해보겠습니다.

## 4. LLaMA-Factory를 활용한 Fine-tuning

### 🛠️ 학습 데이터 준비

이제 **LLaMA-Factory**를 사용하여 이미지 분류 성능을 향상시키기 위한 fine-tuning을 진행합니다.

먼저 다음 명령어를 실행하여 멀티모달 학습용 데이터셋을 생성하겠습니다.

**참고 자료**: [LLaMA-Factory 멀티모달 데이터셋 설정 예시](https://github.com/hiyouga/LLaMA-Factory/blob/main/data/mllm_demo.json)

In [ ]:
# 데이터 생성 코드
python -m demo.create_image_classificat_dataset
# 생성된 데이터 확인
head -20 data/image_classification/train.json

### ⚙️ Fine-tuning 실행

생성된 데이터셋을 확인했으니, 이제 실제 fine-tuning을 시작합니다:

In [ ]:
# demo/image_classification.yaml 파일을 확인해보자.
CUDA_VISIBLE_DEVICES=0,1 llamafactory-cli train demo/image_classification.yaml

### 🚀 Fine-tuned 모델(LoRA) 서빙

학습이 완료되면 vLLM에 **LoRA 어댑터**를 추가하여 fine-tuned 모델을 서빙합니다.

다음 명령어로 LoRA가 적용된 vLLM 서버를 실행하세요:

In [ ]:
export HF_TOKEN=<허깅페이스 토큰>
CUDA_VISIBLE_DEVICES=0,1 vllm serve Qwen/Qwen2.5-VL-7B-Instruct -tp 2 --enable-lora --lora-modules image_classification=$PWD/save/qwen-lora

## 5. 성능 비교 및 평가

### 🔍 Base Model vs Fine-tuned Model 비교

vLLM 서버가 LoRA와 함께 실행되면, 다음 두 모델의 성능을 직접 비교해보겠습니다:

1. **Base Model**: 원본 Qwen2.5-VL-7B-Instruct
2. **Fine-tuned Model**: LoRA가 적용된 모델

동일한 입력에 대한 두 모델의 출력을 비교하여 fine-tuning의 효과를 확인합니다.

In [ ]:
options = ['Mountain', 'Glacier', 'Street', 'Sea', 'Forest', 'Buildings']

constrained_lora_llm = ChatOpenAI(
    base_url="http://localhost:8000/v1",  # vLLM 서버 주소
    api_key="EMPTY",  # vLLM에서는 임의의 값 사용
    model="image_classification",  # Lora 이름
    temperature=0.1,
    extra_body={
        "guided_choice": options
    }
)

# LLM에 요청
response = constrained_lora_llm.invoke(messages)

print(response.content)

### 📏 정량적 성능 평가

학습이 제대로 되었는지 확인하기 위해 `data/image_classification/test.json`을 활용하여 두 모델의 정확도를 비교해보겠습니다.

**평가 방법:**
- 테스트 데이터셋의 각 이미지에 대해 예측 수행
- Base model과 Fine-tuned model의 정확도 측정
- 배치 처리를 통한 효율적인 평가 진행

이를 통해 fine-tuning이 실제로 성능 향상에 기여했는지 확인할 수 있습니다.

In [ ]:
import json
import base64
from PIL import Image
from io import BytesIO
import os
from tqdm import tqdm

# test.json 파일 로드
with open('data/image_classification/test.json', 'r', encoding='utf-8') as f:
    test_data = json.load(f)

print(f"총 테스트 샘플 수: {len(test_data)}")

# 이미지를 base64로 인코딩하는 함수
def encode_image_to_base64(image_path):
    full_path = os.path.join('data', image_path)
    image = Image.open(full_path)
    buffer = BytesIO()
    image.save(buffer, format="JPEG")
    base64_image = base64.b64encode(buffer.getvalue()).decode("utf-8")
    return base64_image

# 배치 예측 함수
def predict_batch_with_model(llm_model, image_paths, system_prompt):
    """배치로 이미지들을 한 번에 처리"""
    batch_messages = []
    
    for image_path in image_paths:
        base64_image = encode_image_to_base64(image_path)
        messages = [
            SystemMessage(content=system_prompt),
            HumanMessage(
                content=[{
                    "type": "image_url", 
                    "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}
                }]
            )
        ]
        batch_messages.append(messages)
    
    # 배치로 한 번에 처리
    responses = llm_model.batch(batch_messages)
    return [response.content.strip() for response in responses]

In [ ]:
# 원본 모델 (베이스라인) 평가
print("=== 원본 모델 (Qwen2.5-VL-7B-Instruct) 평가 ===")

options = ['Mountain', 'Glacier', 'Street', 'Sea', 'Forest', 'Buildings']
system_prompt = "주어진 이미지에 대해서 다음 중 하나로 분류하시오: ['Mountain', 'Glacier', 'Street', 'Sea', 'Forest', 'Buildings']"

# 원본 모델 설정
constrained_llm = ChatOpenAI(
    base_url="http://localhost:8000/v1",
    api_key="EMPTY", 
    model="Qwen/Qwen2.5-VL-7B-Instruct",
    temperature=0.1,
    extra_body={
        "guided_choice": options
    }
)

correct_baseline = 0
total_samples = len(test_data)

# 테스트할 샘플 수 제한 (전체 데이터가 너무 클 경우)
batch_size = 30  # 배치 크기

print(f"평가할 샘플 수: {total_samples}")
print(f"배치 크기: {batch_size}")

# 배치 단위로 처리
for batch_start in tqdm(range(0, total_samples, batch_size), desc="원본 모델 평가 (배치)"):
    batch_end = min(batch_start + batch_size, total_samples)
    batch_samples = test_data[batch_start:batch_end]
    
    # 배치 데이터 준비
    batch_image_paths = [sample['images'][0] for sample in batch_samples]
    batch_true_labels = [sample['messages'][2]['content'] for sample in batch_samples]
    
    # 배치 예측
    batch_predictions = predict_batch_with_model(constrained_llm, batch_image_paths, system_prompt)
    
    # 정확도 계산
    for true_label, predicted_label in zip(batch_true_labels, batch_predictions):
        if predicted_label == true_label:
            correct_baseline += 1
    
baseline_accuracy = correct_baseline / total_samples
print(f"\n원본 모델 최종 정확도: {baseline_accuracy:.4f} ({correct_baseline}/{total_samples})")

In [ ]:
# LoRA 모델 평가
print("\n=== LoRA 모델 (image_classification) 평가 ===")

# LoRA 모델 설정
constrained_lora_llm = ChatOpenAI(
    base_url="http://localhost:8000/v1",
    api_key="EMPTY",
    model="image_classification",  # LoRA 모델 이름
    temperature=0.1,
    extra_body={
        "guided_choice": options
    }
)

correct_lora = 0

# 배치 단위로 처리
for batch_start in tqdm(range(0, total_samples, batch_size), desc="LoRA 모델 평가 (배치)"):
    batch_end = min(batch_start + batch_size, total_samples)
    batch_samples = test_data[batch_start:batch_end]
    
    # 배치 데이터 준비
    batch_image_paths = [sample['images'][0] for sample in batch_samples]
    batch_true_labels = [sample['messages'][2]['content'] for sample in batch_samples]
    
    # 배치 예측
    batch_predictions = predict_batch_with_model(constrained_lora_llm, batch_image_paths, system_prompt)
    
    # 정확도 계산
    for true_label, predicted_label in zip(batch_true_labels, batch_predictions):
        if predicted_label == true_label:
            correct_lora += 1
    
lora_accuracy = correct_lora / total_samples
print(f"\nLoRA 모델 최종 정확도: {lora_accuracy:.4f} ({correct_lora}/{total_samples})")

In [ ]:
# 최종 결과 비교
print("\n" + "="*50)
print("📊 최종 평가 결과")
print("="*50)
print(f"🔹 원본 모델 정확도:  {baseline_accuracy:.4f} ({correct_baseline}/{total_samples})")
print(f"🔸 LoRA 모델 정확도:  {lora_accuracy:.4f} ({correct_lora}/{total_samples})")
print("-"*50)

# 성능 향상도 계산
improvement = lora_accuracy - baseline_accuracy
improvement_percent = (improvement / baseline_accuracy) * 100 if baseline_accuracy > 0 else 0

if improvement > 0:
    print(f"✅ LoRA 모델이 {improvement:.4f} ({improvement_percent:.2f}%) 향상됨")
elif improvement < 0:
    print(f"❌ LoRA 모델이 {abs(improvement):.4f} ({abs(improvement_percent):.2f}%) 하락함")
else:
    print("🟰 두 모델의 성능이 동일함")

print("="*50)

## 6. 결론 및 요약

### 🎯 실습 결과 정리

이번 튜토리얼에서는 다음과 같은 과정을 통해 멀티모달 LLM의 이미지 분류 성능을 향상시켰습니다:

1. **🚀 vLLM 서버 설정**: Qwen2.5-VL-7B-Instruct 모델을 멀티모달 서빙 환경에서 실행
2. **🧪 Zero-shot 평가**: Constrained decoding을 통해 기본 모델의 분류 성능 확인
3. **🛠️ 데이터 준비**: Intel 이미지 분류 데이터셋을 LLaMA-Factory 형식으로 변환
4. **📈 Fine-tuning**: LoRA를 사용한 효율적인 모델 학습
5. **📊 성능 비교**: Base model vs Fine-tuned model의 정량적 평가

### 💡 주요 학습 포인트

- **Constrained Decoding**: 출력을 특정 선택지로 제한하여 일관된 분류 결과 확보
- **멀티모달 Fine-tuning**: 텍스트와 이미지를 동시에 처리하는 모델의 특화 학습
- **LoRA 적용**: 메모리 효율적인 fine-tuning으로 대용량 모델 개선
- **배치 평가**: 효율적인 대량 데이터 처리 및 성능 측정

### 🔧 실무 활용 방안

이 방법론은 다음과 같은 실무 문제에 적용할 수 있습니다:
- **의료 이미지 분류**: 엑스레이, MRI 등의 의료 영상 진단 보조
- **품질 검사**: 제조업에서의 불량품 자동 분류
- **콘텐츠 모더레이션**: 소셜 미디어 이미지 자동 필터링
- **문서 분류**: 스캔된 문서의 자동 카테고리 분류